1. Utilice la herramienta pefile para examinar el PE header y obtenga las DLL y las APIs que cada uno de los ejecutables utilizan. ¿Qué diferencias observa entre los ejemplos? ¿Existe algún indicio sospechoso en la cantidad de DLLs y las APIs llamadas?
2. Obtenga la información de las secciones del PE Header. ¿Qué significa que algunas secciones tengan como parte de su nombre “upx”? Realice el procedimiento de desempaquetado para obtener las llamadas completas de las APIs.
3. Según el paper “Towards Understanding Malware Behaviour by the Extraction of API Calls”, ¿en que categoría sospechosas pueden clasificarse estos ejemplos en base a algunas de las llamadas a las APIs que realizan? Muestre una tabla con las APIs sospechosas y la categoría de malware que el paper propone.
4. Para el archivo “sample_vg655_25th.exe” obtenga el HASH en base al algoritmo SHA256.
5. Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la DLL ADVAPI32.dll?
6. Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la API
CryptReleaseContext?
7. Con la información recopilada hasta el momento, indique para el archivo
““sample_vg655_25th.exe” si es sospechoso o no, y cual podría ser su propósito.

In [5]:
import pefile
pe_sample = pefile.PE("./malware/sample_vg655_25th.exe")
for entry in pe_sample.DIRECTORY_ENTRY_IMPORT:
    print('LLAMADAS A DLL \t')
    print (entry.dll)
    print('LLAMADAS A FUNCIONES \t')
    for function in entry.imports:
        print ('\t', function.name)

LLAMADAS A DLL 	
b'KERNEL32.dll'
LLAMADAS A FUNCIONES 	
	 b'GetFileAttributesW'
	 b'GetFileSizeEx'
	 b'CreateFileA'
	 b'InitializeCriticalSection'
	 b'DeleteCriticalSection'
	 b'ReadFile'
	 b'GetFileSize'
	 b'WriteFile'
	 b'LeaveCriticalSection'
	 b'EnterCriticalSection'
	 b'SetFileAttributesW'
	 b'SetCurrentDirectoryW'
	 b'CreateDirectoryW'
	 b'GetTempPathW'
	 b'GetWindowsDirectoryW'
	 b'GetFileAttributesA'
	 b'SizeofResource'
	 b'LockResource'
	 b'LoadResource'
	 b'MultiByteToWideChar'
	 b'Sleep'
	 b'OpenMutexA'
	 b'GetFullPathNameA'
	 b'CopyFileA'
	 b'GetModuleFileNameA'
	 b'VirtualAlloc'
	 b'VirtualFree'
	 b'FreeLibrary'
	 b'HeapAlloc'
	 b'GetProcessHeap'
	 b'GetModuleHandleA'
	 b'SetLastError'
	 b'VirtualProtect'
	 b'IsBadReadPtr'
	 b'HeapFree'
	 b'SystemTimeToFileTime'
	 b'LocalFileTimeToFileTime'
	 b'CreateDirectoryA'
	 b'GetStartupInfoA'
	 b'SetFilePointer'
	 b'SetFileTime'
	 b'GetComputerNameW'
	 b'GetCurrentDirectoryA'
	 b'SetCurrentDirectoryA'
	 b'GlobalAlloc'
	 b'LoadLibrar

In [6]:
pe_sample2 = pefile.PE("./malware/sample_qwrty_dk2")
for entry in pe_sample2.DIRECTORY_ENTRY_IMPORT:
    print('LLAMADAS A DLL \t')
    print (entry.dll)
    print('LLAMADAS A FUNCIONES \t')
    for function in entry.imports:
        print ('\t', function.name)

LLAMADAS A DLL 	
b'KERNEL32.DLL'
LLAMADAS A FUNCIONES 	
	 b'LoadLibraryA'
	 b'ExitProcess'
	 b'GetProcAddress'
	 b'VirtualProtect'
LLAMADAS A DLL 	
b'MSVCRT.dll'
LLAMADAS A FUNCIONES 	
	 b'atol'
LLAMADAS A DLL 	
b'SHELL32.dll'
LLAMADAS A FUNCIONES 	
	 b'SHChangeNotify'
LLAMADAS A DLL 	
b'USER32.dll'
LLAMADAS A FUNCIONES 	
	 b'LoadStringA'
LLAMADAS A DLL 	
b'WS2_32.dll'
LLAMADAS A FUNCIONES 	
	 b'closesocket'


1. ¿Qué diferencias observa entre los ejemplos? ¿Existe algún indicio sospechoso en la cantidad de DLLs y las APIs llamadas? en En uno se llaman funciones y en el otro se abren archivos. Es muy sospechoso el archivo "sample_vg655_25th" dado a que lee y abre archivos lo cual no debería de pasar.

In [7]:
print('SECCIONES \t')
for section in pe_sample.sections:
    print(section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize),
section.SizeOfRawData)

SECCIONES 	
b'.text\x00\x00\x00' 0x1000 0x69b0 28672
b'.rdata\x00\x00' 0x8000 0x5f70 24576
b'.data\x00\x00\x00' 0xe000 0x1958 8192
b'.rsrc\x00\x00\x00' 0x10000 0x349fa0 3448832


In [8]:
print('SECCIONES \t')
for section in pe_sample2.sections:
    print(section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize),
section.SizeOfRawData)

SECCIONES 	
b'UPX0\x00\x00\x00\x00' 0x1000 0x5000 0
b'UPX1\x00\x00\x00\x00' 0x6000 0x1000 4096
b'.rsrc\x00\x00\x00' 0x7000 0x1000 512


2. Obtenga la información de las secciones del PE Header. ¿Qué significa que algunas secciones tengan como parte de su nombre “upx”? Realice el procedimiento de desempaquetado para obtener las llamadas completas de las APIs.
Quieren decir que están muy comprimidos por lo cual oculta información que nos puede apoyar a ver que tan peligroso es.

3. Según el paper “Towards Understanding Malware Behaviour by the Extraction of API Calls”, ¿en que categoría sospechosas pueden clasificarse estos ejemplos en base a algunas de las llamadas a las APIs que realizan? Muestre una tabla con las APIs sospechosas y la categoría de malware que el paper propone.

In [10]:
import hashlib
hashable = hashlib.sha256()
with open("./malware/sample_vg655_25th.exe", 'rb') as file:
  for byte in iter(lambda: file.read(4096), b""):
    hashable.update(byte)
  print(hashable.hexdigest())

ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa


4. Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la DLL ADVAPI32.dll? La DLL ADVAPI32.dll es una biblioteca de enlace dinámico utilizada por el sistema operativo Microsoft Windows. Contiene una serie de funciones de programación que se utilizan para llevar a cabo diversas tareas relacionadas con la seguridad y la gestión de recursos del sistema. Algunas de las funciones incluidas en la ADVAPI32.dll son: Autenticación y autorización de usuarios, Gestión de claves de registro y criptografía, Administración de servicios y eventos del sistema, Acceso a la información del sistema y la configuración de seguridad, Comunicación entre procesos y usuarios del sistema. En resumen, el propósito principal de la ADVAPI32.dll es proporcionar a los desarrolladores de software de Windows una interfaz para acceder a funciones de seguridad y gestión del sistema operativo de manera eficiente.

5. Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la API
CryptReleaseContext?
La API CryptReleaseContext se utiliza en aplicaciones de Windows para liberar un contexto de criptografía y liberar los recursos del sistema asociados con ese contexto. El contexto de criptografía es un objeto que se crea mediante la API CryptAcquireContext y se utiliza para realizar operaciones criptográficas, como la generación de claves o la encriptación de datos.

6. Con la información recopilada hasta el momento, indique para el archivo
““sample_vg655_25th.exe” si es sospechoso o no, y cual podría ser su propósito. Usa commandos de crypt lo cual sugiere que algo raro está sucediendo.

Parte 2
1. Utilice la plataforma de análisis dinámico https://www.hybrid-analysis.com y cargue el archivo “sample_vg655_25th.exe”. ¿Se corresponde el HASH de la plataforma con el generado? ¿Cuál es el nombre del malware encontrado? ¿Cuál es el propósito de este malware?
- owo_im_not_ransomware_xd.exe
- ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa

Si es igual y el nombre nos sugiere que es un ransomware.

2. Muestre las capturas de pantalla sobre los mensajes que este malware presenta a usuario. ¿Se corresponden las sospechas con el análisis realizado en el punto 7?

Se adjuntan fotos en el repo.

